In [1]:
import numpy as np

In [2]:
import sys
sys.path.insert(0, '/home/adrien/MVA/KERNEL/DataChallenge-KernelMethods/Code/Alex/')

from utils import *
from svm import *

# --Data--

In [3]:
folder = '/home/adrien/MVA/KERNEL/DataChallenge-KernelMethods/Data/'

In [4]:
dataset = 0

In [5]:
data = read_data(folder, dataset=dataset)

In [6]:
print(data.keys())

dict_keys(['Xtr', 'Xte', 'Xtr_mat50', 'Ytr', 'Xte_mat50'])


# --Substring kernel--

In [11]:
def substring_kernel(x, y, gamma, k):
    """
    x, y: input strings
    gamma: real value
    k: length of substring
    
    compute substring kernel distance between x and y
    """
    
    N = len(x)
    M = len(y)
    grid_B = np.zeros((M, N, k+1))
    grid_B[:, :, 0] += np.ones((M, N))
    grid_K = np.zeros((M, N, k+1))
    
    for p in range(1, k+1):
        for i in range(M):
            for j in range(N):
                
                if min(i+1, j+1) < p:
                    continue
                
                sum_B = 0
                sum_K = 0
                
                if p == 1 and j == 0:
                    
                    for n, a in enumerate(y[0:i+1]):
                        
                        if a == x[0]:
                            
                            sum_B += gamma ** (i+1-n)
                            sum_K += gamma ** 2
                    
                    grid_B[i, 0, 1] = gamma * sum_B
                    grid_K[i, 0, 1] = sum_K
                 
                else:
                    
                    for n in range(1, i+1):
                        
                        if y[n] == x[j]:
                            
                            sum_B += grid_B[n-1, j-1, p-1] * (gamma ** (i-n+2))
                            sum_K += grid_B[n-1, j-1, p-1]
                    
                    grid_B[i, j, p] = gamma * grid_B[i, j-1, p] + sum_B
                    grid_K[i, j, p] = grid_K[i, j-1, p] + (gamma ** 2) * sum_K
    
    return grid_K[M-1, N-1, k]

In [12]:
substring_kernel('lpax', 'mnih', 0.8, 2)

0.0

# --Approx kernel--

In [13]:
nb_test = 100

In [14]:
x_test = data['Xtr'][0:nb_test]

In [17]:
alphabet = ['A', 'C', 'T', 'G']

In [27]:
k = 4
l = 0.8

In [115]:
substrings = []

In [116]:
for i1 in alphabet:
    for i2 in alphabet:
        for i3 in alphabet:
            for i4 in alphabet:
                
                if i1+i2+i3+i4 not in substrings:
                    substrings.append(i1+i2+i3+i4)

In [117]:
len(substrings)

256

In [110]:
features = np.zeros((nb_test, len(alphabet)**k))

In [111]:
for i in range(nb_test):
    for j in tqdm(range(len(substrings))):
        
        features[i, j] = substring_kernel(x_test[i], substrings[j], l, k)


 66%|██████▌   | 169/256 [00:00<00:00, 418.88it/s]Exception in thread Thread-207:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/home/adrien/MVA/KERNEL/env3.5/lib/python3.5/site-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/home/adrien/MVA/KERNEL/env3.5/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

100%|██████████| 256/256 [00:00<00:00, 407.41it/s]


100%|██████████| 256/256 [00:00<00:00, 424.67it/s]


In [34]:
K_approx = np.zeros((nb_test, nb_test))
K_real = np.zeros((nb_test, nb_test))

In [70]:
constant = substring_kernel(substrings[0], substrings[0], l, k)

In [71]:
for i in range(nb_test):
    for j in range(i+1, nb_test):
        
        K_approx[i, i] = features[i, :].T.dot(features[i, :]) / constant
        K_approx[i, j] = features[i, :].T.dot(features[j, :]) / constant
        K_approx[j, i] = K_approx[i, j]

In [36]:
for i in range(nb_test):
    for j in tqdm(range(i+1, nb_test)):
        
        K_real[i, i] = substring_kernel(x_test[i], x_test[i], l, k)
        K_real[i, j] = substring_kernel(x_test[i], x_test[j], l, k)
        K_real[j, i] = K_real[i, j]

100%|██████████| 1/1 [00:01<00:00,  1.15s/it]
0it [00:00, ?it/s]


In [37]:
def alignment(K_1, K_2):
    
    if K_1.shape != K_2.shape:
        return False
    
    return np.sum(K_1*K_2) / np.sqrt(np.sum(K_1*K_1) * np.sum(K_2*K_2))

In [69]:
alignment(K_approx, K_real)

0.4716830083077102

In [60]:
K_base = np.zeros((len(substrings), len(substrings)))

In [64]:
for i in tqdm(range(len(substrings))):
    for j in range(i+1, len(substrings)):
        
        K_base[i, i] = substring_kernel(substrings[i], substrings[i], l, k)
        K_base[i, j] = substring_kernel(substrings[i], substrings[j], l, k)
        K_base[j, i] = K_base[i, j]

100%|██████████| 256/256 [00:06<00:00, 41.99it/s]


In [93]:
full_alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 
                 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [103]:
full_substrings = []

In [104]:
for x in full_alphabet:
    for y in full_alphabet:
        
        if x+y not in full_substrings:
            full_substrings.append(x+y)

In [249]:
def substring_explicit(subs, lamb, x, y):
    
    K = 0
    
    for t in tqdm(range(len(subs))):
        
        u = subs[t]
        l_x = []
        l_y = []
        
        for i1 in range(len(x)):
            
            if x[i1] == u[0]:
                
                for i2 in range(i1+1, len(x)):
                    
                    if x[i2] == u[1]:
                        
                        l_x.append(i2-i1+1)
                        
                        """
                        
                        for i3 in range(i2+1, len(x)):
                            
                            if x[i3] == u[2]:
                            
                                for i4 in range(i3+1, len(x)):
                                
                                    if x[i4] == u[3]:
                                        
                                        l_x.append(i4-i1+1)
                        """
        
        for j1 in range(len(y)):
            
            if y[j1] == u[0]:
                
                for j2 in range(j1+1, len(y)):
                    
                    if y[j2] == u[1]:
                        
                        l_y.append(j2-j1+1)
                        
                        """
                        
                        for j3 in range(j2+1, len(y)):
                            
                            if y[j3] == u[2]:
                                
                                for j4 in range(j3+1, len(y)):
                                    
                                    if y[j4] == u[3]:
                                        
                                        l_y.append(j4-j1+1)
                        """
                                                
        for i in l_x:
            for j in l_y:
                
                K += lamb**(i+j)
        
        #print(u, K, l_x, l_y)
        
    return K

In [166]:
substring_explicit(substrings, l, 'ATCCTGAGCTCCACTACTA', 'ATCCTGAGCTCCACTACTA')

100%|██████████| 256/256 [00:00<00:00, 6186.72it/s]


693.6131925382224

In [164]:
substring_kernel('ATCCTGAGCTCCACTACTG', 'ATCCTGAGCTCCACTACTG', l, 4)

570.5955243525651

In [153]:
features[0, :].T.dot(features[0, :]) / constant

94.9593629624136

In [162]:
substring_explicit(substrings, l, 'ATCC', 'ATCC')

100%|██████████| 256/256 [00:00<00:00, 92166.68it/s]


0.1677721600000001

In [163]:
substring_kernel('ATCC', 'ATCC', l, 4)

0.16777216000000014

In [265]:
def K(s, t, gamma, N):
    
    if N < 2:
        print('works only if N > 1')
        return False
    
    S = len(s)
    T = len(t)
    grid = np.ones((S, T, N), dtype=np.float64)
    grid_K = np.zeros((S, T), dtype=np.float64)
    
    #kernel = 0
    
    for p in range(1, N):
        for i in range(S):
            for j in range(T):
                      
                if min(i+1, j+1) < p:
                    grid[i, j, p] = 0

                elif p == 1 and i == 0:

                    sum_gamma = 0

                    for n, a in enumerate(t[0:j+1]):
                        if a == s[0]:
                            sum_gamma += gamma**(j+1-n)

                    grid[i, j, p] = gamma * sum_gamma

                else:

                    sum_gamma = 0

                    for n, a in enumerate(t[0:j+1]):
                        if a == s[i]:
                            sum_gamma += grid[i-1, n-1, p-1] * (gamma**(j-n+2))

                    grid[i, j, p] = gamma * grid[i-1, j, p] + sum_gamma
                    
    """                    
    for i in range(N-1, S):
        
        sum_K = 0
        
        for n, a in enumerate(t):
            sum_K += grid[i-1, n-1, N-1]
            
        kernel = kernel + sum_K * (gamma**2) 
        print(kernel)
                
                
    """     
                
    for i in range(S):
        for j in range(T):         
                
            if min(i+1, j+1) < N:
                continue

            else:

                sum_gamma = 0

                for n, a in enumerate(t[0:j+1]):
                    if a == s[i]:
                        sum_gamma += grid[i-1, n-1, N-1] * (gamma**2)

                grid_K[i, j] = grid_K[i-1, j] + sum_gamma

    
    return grid_K[S-1, T-1]
                        

In [276]:
K('catklbfgiezbkjjgdcjeaehzjazzaahdbevkjhugsdvyrusdbuc', 'azzaahdbevkjhugsdvyrusdbuccatklbfgiezbkjjgdcjeaehzj', 0.8, 2)

146.79833962651549

In [169]:
2*0.8**4 + 0.8**6

1.0813440000000003

In [259]:
l**2

0.6400000000000001

In [275]:
substring_explicit(full_substrings, 0.8, 'catklbfgiezbkjjgdcjeaehzjazzaahdbevkjhugsdvyrusdbuc', 'azzaahdbevkjhugsdvyrusdbuccatklbfgiezbkjjgdcjeaehzj')

100%|██████████| 676/676 [00:00<00:00, 26362.59it/s]


140.42970012608933

# --Main--

parameters

In [6]:
test_size = 0.25
tau = 1e-6
k = 4
gamma = 0.5

data

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X = data['Xtr']
Y = data['Ytr']

In [9]:
X_train, X_test, Y_train, Y_test = (train_test_split(X, Y, test_size=test_size))

model - train

In [11]:
from dev import *

In [ ]:
Kernel = Kernel_Substring(k, gamma)
svm_dual = kernel_SVM(tau, X_train, Y_train, Kernel, dual=True)

model - test

In [ ]:
x_sol, alpha, acc_train = svm_dual.svm_solver(solver='cvxopt')
acc_test = svm_dual.compute_accuracy(X_test, Y_test, alpha)
print('acc_train', acc_train)
print('acc_test', acc_test)
print('DUAL OPTIMIZATION FINISHED')

In [16]:
len(X_train[0])

100